In [2]:
import pandas as pd
import plotly.express as px

In [3]:
D_ALUNO = pd.read_csv("data/refined/d_aluno.csv")
D_CURSO = pd.read_csv("data/refined/d_curso.csv")
D_DISCIPLINA = pd.read_csv("data/refined/d_disciplina.csv")
D_PERIODO = pd.read_csv("data/refined/d_periodo.csv")
F_DESEMPENHO_ACADEMICO = pd.read_csv(
    "data/refined/f_desempenho_academico.csv", low_memory=False
)
F_MATRICULA_ALUNO = pd.read_csv("data/refined/f_matricula_aluno.csv")
# DF_TRATADO = pd.read_csv('data/trusted/arquivo_anonimizado.csv')

# Alunos


## Quantidade de Homens e Mulhres


In [4]:
df_genero = pd.merge(D_ALUNO, F_MATRICULA_ALUNO, on="sk_d_aluno", how="inner")
df_genero = df_genero[
    (df_genero["situacao_matricula"] == "Ativa") & (df_genero["periodo"] == "2021/1") # Corrigir esse filtro do Periodo Atual
]

df_genero = df_genero.groupby(["sexo"]).size().reset_index(name="quantidade")
df_genero.loc[len(df_genero)] = ["Ambos", df_genero["quantidade"].sum()]
df_genero.set_index("sexo")["quantidade"].to_dict()

{'F': 77, 'M': 487, 'Ambos': 564}

## CRA por Periodo


In [5]:
cra_periodo = pd.merge(
    D_PERIODO, F_DESEMPENHO_ACADEMICO, on="sk_d_periodo", how="inner"
)

cra_periodo_filtrado = cra_periodo[
    (cra_periodo["periodo"].str.contains("/1"))
    | (cra_periodo["periodo"].str.contains("/2"))
]

cra_periodo_filtrado = (
    cra_periodo_filtrado.groupby(["periodo"])["cr_acumulado"]
    .mean()
    .reset_index(name="cr_acumulado")
)

## Quantidade de alunos que Evadiram, Trancaram e Concluiram


In [6]:
situacao_aluno = pd.merge(D_ALUNO, F_MATRICULA_ALUNO, on="sk_d_aluno", how="inner")

In [7]:
def gerar_grafico_evação(data, periodo):
    data = data.groupby(['situacao_matricula', periodo]).size().reset_index(name='quantidade')
    data_filtrada = data[(data[periodo].str.contains('/1')) | (data[periodo].str.contains('/2'))]
    data_filtrada = data_filtrada.sort_values(periodo)
    fig = px.bar(data_filtrada, x=periodo, y="quantidade", title=f'Quantidade de Evasão por {periodo.replace('_', ' ').title()}', color='situacao_matricula')
    fig.show()

In [8]:
situacao_aluno.sort_values('periodo', inplace=True)

In [15]:
situacao_aluno[(situacao_aluno['periodo'] == '2019/1') &
               (situacao_aluno['situacao_matricula'] == 'Ativa')]

,sk_d_aluno,nome_completo,matricula_dre,sexo,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,periodo_ingresso_ufrj,periodo_ingresso_curso_atual,periodo,ano,semestre,sk_d_curso
1608,f6c9da6121853fcc02a24a3bc2d82fba916fe4e7362214...,Luiz Henrique Pires,1721,M,Ativa,4,1710,2013/1,2016/1,2019/1,2019,1,42b2581e5ef23445d17a6c6995ee6de021b474b2761100...


In [21]:
gerar_grafico_evação(situacao_aluno, "periodo")

In [22]:
gerar_grafico_evação(situacao_aluno, "periodo_ingresso_ufrj")

In [133]:
situacao_aluno = pd.merge(situacao_aluno, D_CURSO, on="sk_d_curso", how="inner")

In [23]:
data = situacao_aluno.groupby(['situacao_matricula', 'curso_ingresso_ufrj']).size().reset_index(name='quantidade')
data_filtrada = data[(data['curso_ingresso_ufrj'] != 'Bacharelado em Ciência da Computação') & (data['curso_ingresso_ufrj'] != 'Bacharelado em Informática') & (data['curso_ingresso_ufrj'] != 'Ciência da Computação')]
data_filtrada = data_filtrada.sort_values('quantidade', ascending=False)
fig = px.bar(data_filtrada, x='curso_ingresso_ufrj', y="quantidade", title=f'Quantidade de Evasão por {'curso_ingresso_ufrj'.replace('_', ' ').title()}', color='situacao_matricula')
fig.show()

KeyError: 'curso_ingresso_ufrj'

## Quantidade de Reprovações


In [28]:
situacao_disciplina = pd.merge(
    D_DISCIPLINA, F_DESEMPENHO_ACADEMICO, on="sk_d_disciplina", how="inner"
)

In [29]:
situacao_disciplina = situacao_disciplina[~(situacao_disciplina['grau_disciplina'].isna()) & (situacao_disciplina['grau_disciplina'] != 'T') & (situacao_disciplina['grau_disciplina'] != 'I')]
situacao_disciplina['grau_disciplina'] = situacao_disciplina['grau_disciplina'].astype(int)/10

In [30]:
situacao_disciplina = pd.merge(
    D_PERIODO, situacao_disciplina, on="sk_d_periodo", how="inner"
)

In [31]:
situacao_disciplina_agg = situacao_disciplina.groupby(["nome_disciplina", "situacao_disciplina"]).size().reset_index(name="quantidade")
situacao_disciplina_pivot = situacao_disciplina_agg.pivot_table(index="nome_disciplina", columns="situacao_disciplina", values="quantidade", fill_value=0)
situacao_disciplina_pivot["total"] = situacao_disciplina_pivot.sum(axis=1)
situacao_disciplina_sorted = situacao_disciplina_pivot.sort_values("total", ascending=False)

top_20 = situacao_disciplina_sorted.head(30)

fig = px.bar(
    top_20,
    x=top_20.index,
    y=["Aprovado", "Reprovado"],
    title="Top 20 Disciplinas com situação",
)
fig.show()


In [32]:
situacao_disciplina = situacao_disciplina[(situacao_disciplina['periodo'].str.contains('/1')) | (situacao_disciplina['periodo'].str.contains('/2'))]

In [33]:
situacao_disciplina_periodo = situacao_disciplina.groupby(['periodo', 'situacao_disciplina']).size().reset_index(name='quantidade')

In [34]:
situacao_disciplina_periodo

,periodo,situacao_disciplina,quantidade
0,2000/1,Aprovado,261
1,2000/1,Reprovado,64
2,2000/2,Aprovado,449
3,2000/2,Reprovado,130
4,2001/1,Aprovado,561
...,...,...,...
82,2020/2,Aprovado,1416
83,2020/2,Reprovado,150
84,2021/1,Aprovado,1547
85,2021/1,Reprovado,233


In [35]:
teste = situacao_disciplina_periodo[
        situacao_disciplina_periodo["situacao_disciplina"] == "Reprovado"
    ]["quantidade"].tolist()

In [36]:
situacao_disciplina_periodo

fig = px.line(
    situacao_disciplina_periodo,
    x='periodo',
    y='quantidade',
    title="Reprovações e Aprovações por Periodo",
    color='situacao_disciplina'
)
fig.show()

In [154]:
situacao_disciplina[situacao_disciplina['nome_disciplina']== 'Números Inteiros Criptografia']['grau_disciplina'].mean()

5.269862114248195

In [153]:
situacao_disciplina[situacao_disciplina['nome_disciplina']== 'Números Inteiros Criptografia']['grau_disciplina'].median()

5.8

In [155]:
teste = situacao_disciplina[situacao_disciplina['nome_disciplina']== 'Números Inteiros Criptografia']

In [156]:
fig = px.histogram(
    teste,
    x='grau_disciplina',
    title="Disitruição de notas",
    nbins = 20
)
fig.show()

In [49]:
mediana_disciplina = situacao_disciplina.groupby('nome_disciplina')['grau_disciplina'].agg(mediana='median', size='size').reset_index().sort_values('size')
mediana_disciplina = mediana_disciplina[mediana_disciplina['size'] > 500]
mediana_disciplina.head(10)

,nome_disciplina,mediana,size
358,Empreendimento em Informática,7.5,724
113,Calculo Infinitesimal IV,6.0,736
114,Calculo Numerico p/Informatica,7.1,789
636,Informática na Administração,8.1,851
805,Linguagens de Programação,7.9,859
982,Organização de Dados II,7.4,859
917,Modelagem de Sistemas Inform I,8.2,859
233,Cálculo Int e Diferencial IV,5.8,873
160,Computação Concorrente,6.4,939
302,Economia,6.7,949


In [58]:
mediana_disciplina = mediana_disciplina.sort_values('mediana', ascending=False)

In [ ]:
pd.read_json

In [59]:
fig = px.bar(
    mediana_disciplina,
    x='nome_disciplina',
    y='mediana',
    title="",
)
fig.show()